# Bài tập 1 : Bài toán cộng hai vector
**Thông tin sinh viên** :

Hoàng Minh Thanh (18424062)

Jupyter notebook (Online) : https://colab.research.google.com/drive/1ZWDRaKeq9D4uJj2D1x11VXML0vpjpPDD?usp=sharing

# 1. Cài đặt chương trình

### 1.1 Cài đặt CUDA

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ngf5w1t3
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ngf5w1t3
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=0a1d4191fc740353c1fc4238ba375d05dd202df3a42a44194d5441ff4aade149
  Stored in directory: /tmp/pip-ephem-wheel-cache-hm6zqpew/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


### 1.2 Viết chương trình chạy code cộng hai vector

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <chrono>

using namespace std::chrono; 
using namespace std;

__global__ void addVecOnDevice(float *vec1, float *vec2, float *out, int n)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < n)
    {
        out[i] = vec1[i] + vec2[i];
    }
}

void addVecOnHost(float *in1, float *in2, float *out, int n)
{
    for (int i = 0; i < n; i++)
        out[i] = in1[i] + in2[i];
}

int main()
{
    int n = 100000;
    float *vec1, *vec2; // input vector
    float *out; // output vector

    vec1 = (float *)malloc(n * sizeof(float));
    vec2 = (float *)malloc(n * sizeof(float));
    out = (float *)malloc(n * sizeof(float));

    // Setup input values
    srand(time(0));
    for (int i = 0; i < n; i++)
    {
        vec1[i] = static_cast<float>(rand())/static_cast<float>(RAND_MAX);
        vec2[i] = static_cast<float>(rand())/static_cast<float>(RAND_MAX);
    }
    
    for (int i = 0; i < min(n, 10); i++)
    {
        printf("%f, %f \n", vec1[i], vec2[i]);
    }

    // Allocate vector to device memory
    float *d_vec1, *d_vec2, *d_out;
    cudaMalloc(&d_vec1, n * sizeof(float));
    cudaMalloc(&d_vec2, n * sizeof(float));
    cudaMalloc(&d_out, n * sizeof(float));

    // Copy inputs to device
    cudaMemcpy(d_vec1, vec1, n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_vec2, vec2, n * sizeof(float), cudaMemcpyHostToDevice);

    // Launch add() kernel on GPU
    dim3 blockSize(256);
    dim3 gridSize((n - 1) / blockSize.x + 1);

    // cudaEvent_t start, stop;
    // cudaEventCreate(&start);
    // cudaEventCreate(&stop);
    // cudaEventRecord(start);
 
    auto start_host = high_resolution_clock::now();
    addVecOnHost(vec1,vec2, out, n);
    auto stop_host = high_resolution_clock::now();
    auto duration_host = duration_cast<microseconds>(stop_host - start_host);
    printf("Time host : %d milliseconds\n", duration_host.count());

    auto start_device = high_resolution_clock::now();
    addVecOnDevice<<<gridSize, blockSize>>>(d_vec1, d_vec2, d_out, n);
    cudaDeviceSynchronize();
    // Copy result back to host
    cudaMemcpy(out, d_out, n * sizeof(float), cudaMemcpyDeviceToHost);
    auto stop_device = high_resolution_clock::now();
    auto duration_device = duration_cast<microseconds>(stop_device - start_device);
    printf("Time device : %d milliseconds\n", duration_device.count());
    
    // cudaEventRecord(stop);
    // cudaEventSynchronize(stop);
    // float milliseconds = 0;
    // cudaEventElapsedTime(&milliseconds, start, stop);
    // printf("Time device : %d milliseconds\n", milliseconds);

    // Cleanup
    cudaFree(d_vec1);
    cudaFree(d_vec2);
    cudaFree(d_out);
 
    free(vec1);
    free(vec2);
    free(out);
 
    return 0;
}

0.130444, 0.676497 
0.700360, 0.977689 
0.182429, 0.528489 
0.302907, 0.783266 
0.569193, 0.239493 
0.295245, 0.263475 
0.648729, 0.311683 
0.376001, 0.995230 
0.862447, 0.923292 
0.743194, 0.064680 
Time host : 472 milliseconds
Time device : 158 milliseconds



### 1.3 Cài đặt phương đo tốc độ cộng hai vector theo kích thước mảng

Trên là ta đã thử xây dựng chương trình tính tổng hai vector

Dưới đây ta sẽ xây dựng bảng để có thể so sánh dễ hơn

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <chrono>
#include <vector>

using namespace std::chrono; 
using namespace std;

__global__ void addVecOnDevice(float *vec1, float *vec2, float *out, int n)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < n)
    {
        out[i] = vec1[i] + vec2[i];
    }
}

void addVecOnHost(float *in1, float *in2, float *out, int n)
{
    for (int i = 0; i < n; i++)
        out[i] = in1[i] + in2[i];
}

void calcTimeWithNSize(int n)
{
    float *vec1, *vec2; // input vector
    float *out; // output vector

    vec1 = (float *)malloc(n * sizeof(float));
    vec2 = (float *)malloc(n * sizeof(float));
    out = (float *)malloc(n * sizeof(float));

    // Setup input values
    srand(time(0));
    for (int i = 0; i < n; i++)
    {
        vec1[i] = static_cast<float>(rand())/static_cast<float>(RAND_MAX);
        vec2[i] = static_cast<float>(rand())/static_cast<float>(RAND_MAX);
    }

    // Allocate vector to device memory
    float *d_vec1, *d_vec2, *d_out;
    cudaMalloc(&d_vec1, n * sizeof(float));
    cudaMalloc(&d_vec2, n * sizeof(float));
    cudaMalloc(&d_out, n * sizeof(float));

    // Copy inputs to device
    cudaMemcpy(d_vec1, vec1, n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_vec2, vec2, n * sizeof(float), cudaMemcpyHostToDevice);

    // Launch add() kernel on GPU
    dim3 blockSize(256);
    dim3 gridSize((n - 1) / blockSize.x + 1);
 
    auto start_host = high_resolution_clock::now();
    addVecOnHost(vec1,vec2, out, n);
    auto stop_host = high_resolution_clock::now();
    auto duration_host = duration_cast<microseconds>(stop_host - start_host);

    auto start_device = high_resolution_clock::now();
    addVecOnDevice<<<gridSize, blockSize>>>(d_vec1, d_vec2, d_out, n);
    cudaDeviceSynchronize();
    // Copy result back to host
    cudaMemcpy(out, d_out, n * sizeof(float), cudaMemcpyDeviceToHost);
    auto stop_device = high_resolution_clock::now();
    auto duration_device = duration_cast<microseconds>(stop_device - start_device);

    // Cleanup
    cudaFree(d_vec1);
    cudaFree(d_vec2);
    cudaFree(d_out);
 
    free(vec1);
    free(vec2);
    free(out);

    printf("|%d ms\t\t|%d ms\t\t|%d ms\t\t|\n", n, duration_host.count(), duration_device.count());
}

int main()
{
    printf("|Size\t\t|Host\t\t|Device\t|\n");
    for (int i = 64; i < 100000000; i = i * 4){
        calcTimeWithNSize(i);
    }
    
    return 0;
}


|Size		|Host		|Device	|
|64 ms		|0 ms		|35 ms		|
|256 ms		|0 ms		|23 ms		|
|1024 ms		|2 ms		|26 ms		|
|4096 ms		|12 ms		|26 ms		|
|16384 ms		|50 ms		|39 ms		|
|65536 ms		|214 ms		|94 ms		|
|262144 ms		|851 ms		|318 ms		|
|1048576 ms		|3462 ms		|867 ms		|
|4194304 ms		|19048 ms		|3471 ms		|
|16777216 ms		|83563 ms		|13723 ms		|
|67108864 ms		|317027 ms		|54897 ms		|



# 2. Bảo cáo

Đề cài đặt chương trình chạy trên GPU thì ta cần :
1. Khởi tạo dữ liệu ltrên host (CPU)
2. Khởi tạo bộ nhớ cho các biến tính toán trên device (GPU)
3. Copy dữ liệu từ host sang device
4. Thực hiện goi hàm tính toán trên device
5. Sau khi thực hiện xong thì copy kết quả từ device sang host
6. Xuất kết quả từ host

#### Nhận xét :
Có thể thấy khi xử lý mảng với số vector dưới 10.000 phần tử thì xử lý trên host tương đương hoặc nhanh hơn so với trên GPU, nhưng khi kích thước mảng càng lớn thì xử lý với tốc độ trên GPU sẽ nhanh hơn so với tốc độ trên GPU.